In [ ]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Activation
from tensorflow.keras.optimizers import RMSprop


In [ ]:

from google.colab import files

uploaded = files.upload()

for filepath in uploaded.keys():
    print(f'Uploaded file "{filepath}" with {len(uploaded[filepath])} bytes')

Saving ALL_eminem.txt to ALL_eminem.txt
Uploaded file "ALL_eminem.txt" with 925480 bytes


In [ ]:
text=open(filepath,'rb').read().decode(encoding='utf8').lower()
# converting the text into numerical form
text=text[100000:1115394]

In [ ]:
characters=sorted(set(text))# filters out all the words
print(f'{len(characters)} unique characters')


82 unique characters


In [ ]:
len(text)

825006

In [ ]:
char_to_index= dict((c,i) for i,c in enumerate(characters))#creating a dictionary where every character have values/index

In [ ]:
index_to_char=dict((i,c) for i,c in enumerate(characters))

In [ ]:
# creating emoty list of entence

SEQ_LENGTH=40
STEP_SIZE=3

sentences=[]
next_characters=[]


for i in range(0,len(text) - SEQ_LENGTH, STEP_SIZE):
  sentences.append(text[i:i+SEQ_LENGTH])
  next_characters.append(text[i+SEQ_LENGTH])




In [ ]:

print(next_characters)

['l', ' ', 'b', 'c', 'a', ' ', 'a', ' ', 'h', 't', 's', 'a', 's', 'p', 'e', 't', 'b', 'm', 'c', 'c', ' ', 't', 'e', 'i', 'l', 't', 't', 'o', 'h', 'r', 'n', 'o', ' ', 'd', '\n', 't', 'h', "'", 'a', 'u', 't', 'h', ' ', 'e', 'h', 'g', '\n', ' ', ' ', ' ', ' ', ' ', ' ', '\n', 'r', 'o', 'a', 'e', 'a', 'h', 'h', 'r', ' ', ' ', '\n', 'r', 'o', 'o', 'a', 'a', 'o', 'a', 'o', 'a', 'o', 'a', 'o', 'a', 'p', ' ', 'o', 'a', 'a', 'a', 'i', 'n', ' ', 'm', 'i', 's', 'h', 'g', 'm', ' ', 't', 'l', 'y', 'm', 'e', 'e', 'e', 'o', 'o', 'r', 'h', 'n', '\n', 't', 'e', ' ', 'a', 'd', 'd', 'a', 'a', ' ', 'l', 'y', '?', 't', 'i', 't', ' ', ' ', 't', 'e', 'o', 'i', '\n', 'i', 'e', 'u', 't', 't', 'p', 'r', 'i', 'w', 't', 'o', 'c', 'i', ' ', 'o', '?', 'o', 'g', ' ', '.', 'h', 'l', ',', ' ', 'o', 'y', ' ', 's', 'y', 'r', 'o', ' ', 'd', ' ', 'o', 'y', ' ', 's', 'y', 'r', 'a', 'w', 'n', "'", 'g', 'e', 'u', 'i', ' ', 'y', 'g', 'o', 'i', ' ', 'u', 'h', 'l', 'e', 'h', ' ', 'n', 'e', 'h', '\n', 'c', ' ', 'e', 'o', 'r', 's

In [ ]:
import numpy as np
x_shape = (len(sentences), SEQ_LENGTH, len(characters))

# Initialize x and y arrays
x = np.zeros(x_shape, dtype=bool)
y_shape=(len(sentences), len(characters))
y = np.zeros(y_shape , dtype=bool)

for i, sentence in enumerate(sentences):
    for t, character in enumerate(sentence):
        x[i, t, char_to_index[character]] = 1
    y[i, char_to_index[next_characters[i]]] = 1



In [ ]:
from keras.optimizers import RMSprop

model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters)), return_sequences=True))
model.add(LSTM(64,return_sequences=True))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(16))
model.add(Dense(len(characters)))
model.add(Activation('Softmax'))

model.compile(loss='categorical_crossentropy',optimizer= RMSprop(learning_rate=0.01))

model.fit(x,y,batch_size=128,epochs=5)
model.save('textgenerator.model')# here we are training the model and saving it to textgenerator.model so that we do not need to train it every time


Epoch 1/5
2149/2149 [==============================] - 309s 141ms/step - loss: 3.0735
Epoch 2/5
2149/2149 [==============================] - 308s 143ms/step - loss: 2.3767
Epoch 3/5
2149/2149 [==============================] - 304s 142ms/step - loss: 1.9974
Epoch 4/5
2149/2149 [==============================] - 309s 144ms/step - loss: 1.8658
Epoch 5/5
2149/2149 [==============================] - 308s 143ms/step - loss: 1.7891


In [ ]:
model=tf.keras.models.load_model('textgenerator.model')

In [ ]:
#converting the prediction into senteces
# higher the temperature higher the risker words chooosed
def sample(preds,temperature=1.0):
  preds=np.asarray(preds).astype('float64')
  preds=np.log(preds)/temperature
  exp_preds=np.exp(preds)
  preds =exp_preds/np.sum(exp_preds)
  probas=np.random.multinomial(1,preds,1)
  return np.argmax(probas)

def generate_text(length,temperature):
  start_index=random.randint(0,len(text) - SEQ_LENGTH - 1)
  generated=''
  sentence=text[start_index:start_index+SEQ_LENGTH]
  generated+=sentence
  for i in range(length):
    x=np.zeros((1,SEQ_LENGTH,len(characters)))
    for t,character in enumerate(sentence):
      x[0,t,char_to_index[character]]=1

    predictions=model.predict(x,verbose=0)[0]
    next_index=sample(predictions,temperature)
    next_character=index_to_char[next_index]

    generated+=next_character
    sentence=sentence[1:]+next_character
  return generated
print('-----0.2--------')
print(generate_text(300,0.2))

print('-----0.4--------')
print(generate_text(300,0.4))

print('-----0.6--------')
print(generate_text(300,0.6))

print('-----0.8--------')
print(generate_text(300,0.8))

print('-----1--------')
print(generate_text(300,1))



-----0.2--------
 one else to see
'cause i feel like i'm ending to say it was good
so i say a crange in the beal with the sack ounks to say i don't know you know i want me where i suck the chustion and the more and say it and say you go and a baman and say i been to and a slim and so it's a mome and here and singer the broke and the should the should have
-----0.4--------
e land of criminal
my era, my era, my eryed to could core this and be all here and one it wanna know the moming in mak to the care
my mouth
i don't wanna get the trasher me whatertord to get a bull and the work and my conger all with a manse
i'm i one on the crack to remander to say this so i'm bust and cryed to the door
here it's so i do
-----0.6--------

proof, kyu and denaun, we move through the loxe ass with in a gecked
pased her bed while me to get a song to get a limy, that you suck that's a chesers, i'mhe greom white slim hipperbapne and in anying to song done and senaesantion
in a making is reded 
bueping for 